In [1]:
# Model
import agentpy as ap
import matplotlib.animation

# Visualization
import matplotlib.pyplot as plt
import IPython

class Interseccion(ap.Model):

    def setup(self):
        # Create cars
        cars = self.cars = ap.AgentList(self, self.p.n_cars)

        # Create streets
        street_length = self.p.size
        self.h_pos = street_length // 2
        self.v_pos = street_length // 2

        segmento_vertical = self.segmento_vertical = ap.AgentList(self, street_length)
        segmento_horizontal = self.segmento_horizontal = ap.AgentList(self, street_length)

        # Create lights
        semaforos = self.semaforos = ap.AgentList(self, 2)

        # Create grid (calles)
        self.streets = ap.Grid(self, (self.p.size, self.p.size), track_empty=True)

        # Posiciones de segmentos
        posicionesV = []
        posicionesH = []
        for i in range(street_length):
            posicionesV.append((street_length//2, i))
            posicionesH.append((i, street_length//2))

        self.streets.add_agents(segmento_vertical, posicionesV)
        self.streets.add_agents(segmento_horizontal, posicionesH)

        posicionesSemaforos = []
        posicionesSemaforos.append((street_length//2-1, street_length//2))
        posicionesSemaforos.append((street_length//2, street_length//2-1))
        self.streets.add_agents(semaforos, posicionesSemaforos)

        # Conditions
        self.segmento_vertical.condition = 0
        self.segmento_horizontal.condition = 0
        self.semaforos.condition = 1

    def step(self):

        """
        self.add_car()

        # mover el carro
        for car in self.cars_in:
            for neighbor in self.streets.neighbors(car):
                if neighbor.condition == 0:
                    self.streets.move_to(car, self.streets.positions[neighbor])
                    break

        self.stop()
        """

        self.stop()

    def end(self):
        pass


def animation_plot(model, ax):
    attr_grid = model.streets.attr_grid('condition')
    color_dict = {0: '#0f0f0f00', 1: '#00ff00', 2: '#0000ff', None: '#eeeeee'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of street intersection\n"
                 f"Time-step: {model.t}, Trees left: ")

parameters = {
    'size': 10,
    'n_cars': 2
}

fig, ax = plt.subplots()
model = Interseccion(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))